# **Binomial Options Pricing Model** <br>
This file will serve as a reference to implementing the binomial options pricing model for European Options

## **Binomial Tree - Variables, Equations** - European Call<br>
If we consider the binomial tree to be nodes that can be represented by vectors we can say:

Stock Price at Node i,j: $$S_i,_j = S_0 \cdot u^j \cdot d^{i-j}$$

Consider a European call option. The option contract price is given by $C_{i,j}$. For a given node, the options contract price at that node is given by
$$C_{i,j} = max(S_{i,j} - K, 0)$$

To hedge the short position, buy $\beta$ shares of stock. As a result, consider $\alpha$ amount of money from the monkey market with risk free interest rate r. $\alpha$ is hence given by $$ \alpha = C_0 - \beta S_0$$

Because we are considering a one period binomial model, we consider 2 cases:

(1) The stock goes up. This leads to this formula: $$C_u = \alpha \cdot e^{rT} + \beta \cdot S_u$$
(2) The stock goes down. This leads to this formula: $$C_d = \alpha \cdot e^{rT} + \beta \cdot S_d$$

Since we want to neutralise our risk, we want to buy $\beta$ shares of stock such that we do not lose money regardless of if the stock goes up or down. So we have to say that the portfolio value of our hedge must be equal for both scenarios.

Hence subtracting both equations, we get that we should buy $\beta$ shares given by: $$\beta = \frac{C_u - C_d}{S_u - S_d}$$

By substituting $\beta$ into either equation (1) or (2) we get the arbitrage price for the European call option with one period: $$ C_0 = e^{-rT}[p C_u + q C_d] $$ 
where $p = \frac{e^{rT} - d}{u - d}$ and $q = 1 - p$ which is the risk-neutral probability.

NB: $S_u = S_0 \cdot u$ and $S_d = S_0 \cdot d$

In [11]:
import numpy as np

In [12]:
# defining variables
S0 = 100    # initial stock price
K = 103     # strike price
T = 1       # time to maturity (years)
N = 3       # no. of time-steps
r = 0.06    # risk-free annual interest rate
u = 1.2     # up factor
d = 1/u     # down factor (inverse for recombining tree)
opt = 'C'   # options type (C - call ; P - put)

## **Binomial Tree - for loops (Slow)**
In this model, we will iterate through the notes using for loops which is slow. Later we will see how to speed this up using numpy vectorisation. 

The strategy is to first compute the contract values at the terminal node layer and work backwards till the initial contract value which will give us the fair options price.

In [13]:
def binomial_slow(S0,K,T,N,r,u,d,opt='C'):
    # first compute equation constants
    dt = T/N 
    p = (np.exp(r*dt) - d)/(u - d)
    discount = np.exp(-r * dt)

    # numpy array to store the stock values at maturity
    S = np.zeros(N+1)
    S[0] = d**N * S0
    for j in range(1,N+1):
        S[j] = S0 * (u ** j) * (d ** (N-j))

    # numpy array to store options values at maturity'
    C = np.zeros(N+1)
    for j in range(0,N+1):
        C[j] = max(0,S[j] - K)
    
    # now let's move backwards through the tree
    for i in np.arange(N,0,-1):
        for j in range(0,i):
            # replace the cell in the nparray with the backwards contract calculation of itself and the one above
            # essentially knocking off the top element            
            C[j] = discount * (p * C[j+1] + (1-p) * C[j]) 
    
    return C[0]

binomial_slow(S0,K,T,N,r,u,d,opt='C')

np.float64(14.81861039129543)